In Python, a metaclass is a class whose instances are classes. Just as an ordinary class defines the behavior of certain objects, a metaclass allows for the customization of class instantiation.

Metaclasses usually enter the game when we program advanced modules or frameworks, where a lot of precise automation must be provided. 

The typical use cases for metaclasses:
- logging;
- registering classes at creation time;
- interface checking;
- automatically adding new methods;
- automatically adding new variables.

The functionality of the metaclass partly coincides with that of class decorators, but metaclasses act in a different way than decorators:

- decorators bind the names of decorated functions or classes to new callable objects. Class decorators are applied when classes are instantiated;
- metaclasses redirect class instantiations to dedicated logic, contained in metaclasses. Metaclasses are applied when class definitions are read to create classes, well before classes are instantiated.


In [3]:
# In Python's approach, everything is an object, 
# and every object has some type associated with it. 
# To get the type of any object, make use of the type() function.
class Dog:
    pass


age = 10
codes = [33, 92]
dog = Dog()

print(type(age))
print(type(codes))
print(type(dog))
print(type(Dog))

for t in (int, list, type):
    print(type(t))

<class 'int'>
<class 'list'>
<class '__main__.Dog'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


In [6]:
# We should get familiar with some special attributes:

# __name__ – inherent for classes; 
#               contains the name of the class;
# __class__ – inherent for both classes and instances; 
#               contains information about the class to which a class instance belongs;
# __bases__ – inherent for classes; 
#               it’s a tuple and contains information about the base classes of a class;
#  __dict__ – inherent for both classes and instances; 
#               contains a dictionary (or other type mapping object) of the object's attributes.
class Dog:
    pass

dog = Dog()
print('"dog" is an object of class named:', Dog.__name__)
print()
print('class "Dog" is an instance of:', Dog.__class__)
print('instance "dog" is an instance of:', dog.__class__)
print()
print('class "Dog" is  ', Dog.__bases__)
print()
print('class "Dog" attributes:', Dog.__dict__)
print('object "dog" attributes:', dog.__dict__)


"dog" is an object of class named: Dog

class "Dog" is an instance of: <class 'type'>
instance "dog" is an instance of: <class '__main__.Dog'>

class "Dog" is   (<class 'object'>,)

class "Dog" attributes: {'__module__': '__main__', '__dict__': <attribute '__dict__' of 'Dog' objects>, '__weakref__': <attribute '__weakref__' of 'Dog' objects>, '__doc__': None}
object "dog" attributes: {}


In [7]:
# The same information stored in __class__could be retrieved by calling a type() function 
# -> with one argument!! :
for element in (1, 'a', True):
    print(element, 'is', element.__class__, type(element))

1 is <class 'int'> <class 'int'>
a is <class 'str'> <class 'str'>
True is <class 'bool'> <class 'bool'>


In [8]:
# When the type() function is called with three arguments, 
# then it dynamically creates a new class.
Dog = type('Dog', (), {}) # name, bases, dict

print('The class name is:', Dog.__name__)
print('The class is an instance of:', Dog.__class__)
print('The class is based on:', Dog.__bases__)
print('The class attributes are:', Dog.__dict__)

The class name is: Dog
The class is an instance of: <class 'type'>
The class is based on: (<class 'object'>,)
The class attributes are: {'__module__': '__main__', '__dict__': <attribute '__dict__' of 'Dog' objects>, '__weakref__': <attribute '__weakref__' of 'Dog' objects>, '__doc__': None}


In [9]:
# The more complex example that dynamically creates a fully functional class is presented in the right pane.
def bark(self):
    print('Woof, woof')

class Animal:
    def feed(self):
        print('It is feeding time!')

Dog = type('Dog', (Animal, ), {'age':0, 'bark':bark})

print('The class name is:', Dog.__name__)
print('The class is an instance of:', Dog.__class__)
print('The class is based on:', Dog.__bases__)
print('The class attributes are:', Dog.__dict__)

doggy = Dog()
doggy.feed()
doggy.bark()

The class name is: Dog
The class is an instance of: <class 'type'>
The class is based on: (<class '__main__.Animal'>,)
The class attributes are: {'age': 0, 'bark': <function bark at 0x736631ed77e0>, '__module__': '__main__', '__doc__': None}
It is feeding time!
Woof, woof


Now that we know what’s happening under Python's hood, it’s time to implement our own metaclass.

It’s important to remember that metaclasses are classes that are instantiated to get classes.

The first step is to define a metaclass that derives from the type type and arms the class with a 'custom_attribute'.

In [10]:
class My_Meta(type): # This makes our class a metaclass;
    def __new__(mcs, name, bases, dictionary):
        obj = super().__new__(mcs, name, bases, dictionary) # 'mcs' to refer to the class – it’s just a convention
        obj.custom_attribute = 'Added by My_Meta' # additionally
        return obj # class returned

class My_Object(metaclass=My_Meta): # way to tell Python to use My_Meta as a metaclass, not as an ordinary superclass
    pass

print(My_Object.__dict__)

{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'My_Object' objects>, '__weakref__': <attribute '__weakref__' of 'My_Object' objects>, '__doc__': None, 'custom_attribute': 'Added by My_Meta'}


In [11]:
def greetings(self):
    print('Just a greeting function, but it could be something more serious like a check sum')

class My_Meta(type):
    def __new__(mcs, name, bases, dictionary):
        if 'greetings' not in dictionary:
            dictionary['greetings'] = greetings
        obj = super().__new__(mcs, name, bases, dictionary)
        return obj

class My_Class1(metaclass=My_Meta):
    pass 
# no greetings function, so when the class is constructed,
# it is equipped with a default function by the metaclass

class My_Class2(metaclass=My_Meta):
    def greetings(self): # function present from the very beginning
        print('We are ready to greet you!')

myobj1 = My_Class1()
myobj1.greetings()
myobj2 = My_Class2()
myobj2.greetings()

Just a greeting function, but it could be something more serious like a check sum
We are ready to greet you!


**Scenario**


- Imagine you’ve been given a task to clean up the code of a system developed in Python – the code should be treated as legacy code;
- the system was created by a group of volunteers who worked with no clear “clean coding” rules;
- the system suffers from a problem: we don’t know in which order the classes are created, so it causes multiple dependency problems;
- your task is to prepare a metaclass that is responsible for:

> equipping all newly instantiated classes with time stamps, persisted in a class attribute named instantiation_time;
        
> equipping all newly instantiated classes with the get_instantiation_time() method. The method should return the value of the class attribute instantiation_time.

* The metaclass should have its own class variable (a list) that contains a list of the names of the classes instantiated by the metaclass (tip: append the class name in the __new__ method).

- Your metaclass should be used to create a few distinct legacy classes;
- create objects based on the classes;
- list the class names that are instantiated by your metaclass.
